In [ ]:
# ============================================================================
# EXEMPLO 2: OTIMIZAÇÃO CONTÍNUA DE THRESHOLD COM BUSCA TERNÁRIA
# Modelagem Matemática Aplicada à Otimização em Análise de Dados
# Prof. Maurizio Prizzi - CEUB
# ============================================================================

"""
OBJETIVO: Demonstrar como otimizar continuamente o threshold de classificação
para adaptação em tempo real a mudanças nos padrões de fraude (concept drift).

CONCEITOS MATEMÁTICOS:
- Busca ternária para otimização de função unimodal
- Otimização contínua e adaptação online
- Análise de sensibilidade de parâmetros
- Convergência de algoritmos de busca
- Simulação de concept drift

"""

# Importando bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize_scalar
import time
import logging
from functools import lru_cache
import warnings
warnings.filterwarnings('ignore')

# Configurando logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

logger.info("=" * 80)
logger.info("EXEMPLO 2: OTIMIZAÇÃO CONTÍNUA DE THRESHOLD COM BUSCA TERNÁRIA")
logger.info("Adaptação em tempo real")
logger.info("=" * 80)

# ============================================================================
# 1. GERAÇÃO DE DATASET E TREINAMENTO DE MODELO BASE
# ============================================================================

def criar_modelo_pre_treinado():
    """
    Cria e treina um modelo base para demonstrar a otimização de threshold
    Simula um modelo já em produção que precisa de calibração contínua
    """
    logger.info("Criando modelo base para otimização de threshold...")

    # Dataset sintético para modelo base
    X, y = make_classification(
        n_samples=12000,
        n_features=15,
        n_informative=12,
        n_clusters_per_class=3,
        weights=[0.95, 0.05],  # 95% legítimas, 5% fraudes
        random_state=42
    )

    # Nomes das features para modelo de fraude
    feature_names = [
        'valor_log', 'velocidade_transacao', 'freq_horaria', 'desvio_comportamental',
        'score_dispositivo', 'historico_localizacao', 'padrão_temporal',
        'network_risk', 'account_age', 'transaction_count', 'velocity_score',
        'geo_anomaly', 'device_fingerprint', 'behavioral_score', 'risk_aggregate'
    ]

    df = pd.DataFrame(X, columns=feature_names)
    df['fraude'] = y

    # Divisão e treinamento
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )

    # Modelo Random Forest pré-configurado
    modelo = RandomForestClassifier(
        n_estimators=200,
        max_depth=20,
        min_samples_split=5,
        random_state=42,
        n_jobs=-1
    )

    # Treinamento
    modelo.fit(X_train, y_train)

    # Probabilidades para otimização de threshold
    y_proba = modelo.predict_proba(X_test)[:, 1]

    logger.info(f"Modelo base treinado:")
    logger.info(f"   Dataset: {len(df):,} transações")
    logger.info(f"   Conjunto teste: {len(X_test):,} transações")
    logger.info(f"   Taxa de fraude: {sum(y_test)/len(y_test)*100:.1f}%")

    return y_test, y_proba, df, X_test, y_train, X_train

# Criando modelo e obtendo dados para otimização
y_true, probabilidades, dados_completos, X_test, y_train, X_train = criar_modelo_pre_treinado()

# ============================================================================
# 2. FUNÇÃO OBJETIVO PARA OTIMIZAÇÃO DE THRESHOLD
# ============================================================================

def calcular_metricas_negocio(threshold, y_true, y_proba):
    """
    Calcula métricas de negócio para um threshold específico

    Parâmetros:
    - threshold: Ponto de corte para classificação (0 < threshold < 1)
    - y_true: Labels verdadeiros
    - y_proba: Probabilidades preditas pelo modelo

    Retorna: Dicionário com métricas calculadas
    """
    # Validação de entrada
    if not (len(y_true) == len(y_proba) and len(y_true) > 0):
        raise ValueError("y_true e y_proba devem ter o mesmo tamanho e não podem ser vazios")
    if not np.all((0 <= y_proba) & (y_proba <= 1)):
        raise ValueError("y_proba deve conter valores entre 0 e 1")
    if not np.all(np.isin(y_true, [0, 1])):
        raise ValueError("y_true deve conter apenas 0 ou 1")

    # Aplicando threshold para gerar predições binárias
    y_pred = (y_proba >= threshold).astype(int)

    # Métricas fundamentais
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    # Matriz de confusão
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    # Métricas derivadas com regularização
    denominador = tn + fp + 1e-10  # Evita divisão por zero
    especificidade = tn / denominador
    fpr = fp / denominador
    fnr = fn / (fn + tp + 1e-10)

    return {
        'threshold': threshold,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'especificidade': especificidade,
        'fpr': fpr,
        'fnr': fnr,
        'tp': tp, 'fp': fp, 'tn': tn, 'fn': fn
    }

@lru_cache(maxsize=1000)
def funcao_objetivo_threshold(threshold, y_true_tuple, y_proba_tuple,
                             peso_recall=0.6, peso_precision=0.25, peso_experiencia=0.15):
    """
    Função objetivo para otimizar threshold baseada em critérios de negócio

    Combina múltiplos objetivos:
    - Maximizar recall (detectar fraudes)
    - Maximizar precisão (evitar falsos alarmes)
    - Maximizar experiência do usuário (baixo FPR)

    Retorna: Valor a ser MAXIMIZADO
    """
    # Converter tuplas de volta para arrays
    y_true = np.array(y_true_tuple)
    y_proba = np.array(y_proba_tuple)

    # Calculando métricas
    metricas = calcular_metricas_negocio(threshold, y_true, y_proba)

    # Score combinado usando pesos de negócio
    score = (peso_recall * metricas['recall'] +
             peso_precision * metricas['precision'] +
             peso_experiencia * metricas['especificidade'])

    return score

# ============================================================================
# 3. IMPLEMENTAÇÃO DA BUSCA TERNÁRIA
# ============================================================================

def avaliar_pontos_ternarios(left, right, y_true, y_proba, peso_recall, peso_precision, peso_experiencia):
    """
    Avalia os pontos de divisão ternária e retorna valores da função objetivo
    """
    delta = (right - left) / 3
    m1 = left + delta
    m2 = right - delta
    f_m1 = funcao_objetivo_threshold(m1, tuple(y_true), tuple(y_proba),
                                     peso_recall, peso_precision, peso_experiencia)
    f_m2 = funcao_objetivo_threshold(m2, tuple(y_true), tuple(y_proba),
                                     peso_recall, peso_precision, peso_experiencia)
    return m1, m2, f_m1, f_m2

def busca_ternaria_otimizada(y_true, y_proba, peso_recall=0.6, peso_precision=0.25,
                             peso_experiencia=0.15, tolerancia=1e-7, max_iteracoes=100):
    """
    Implementa busca ternária para encontrar threshold ótimo

    A busca ternária é eficiente para funções unimodais (um único máximo global)
    Complexidade: O(log₃(n)) onde n é o tamanho do espaço de busca

    Parâmetros:
    - tolerancia: Precisão desejada para convergência
    - max_iteracoes: Número máximo de iterações

    Retorna: threshold_otimo, score_otimo, historico_convergencia
    """
    logger.info("Iniciando Busca Ternária para Threshold Ótimo...")
    logger.info(f"   Tolerância: {tolerancia}")
    logger.info(f"   Máx. iterações: {max_iteracoes}")

    # Intervalo inicial de busca
    left = 0.001
    right = 0.999

    # Histórico para análise de convergência
    historico = []

    inicio_tempo = time.time()

    for iteracao in range(max_iteracoes):
        # Avaliar pontos ternários
        m1, m2, f_m1, f_m2 = avaliar_pontos_ternarios(
            left, right, y_true, y_proba, peso_recall, peso_precision, peso_experiencia
        )

        # Salvando histórico
        historico.append({
            'iteracao': iteracao + 1,
            'left': left,
            'right': right,
            'm1': m1,
            'm2': m2,
            'f_m1': f_m1,
            'f_m2': f_m2,
            'intervalo_size': right - left
        })

        # Atualização do intervalo
        if f_m1 > f_m2:
            right = m2
        else:
            left = m1

        # Critério de convergência
        if (right - left) < tolerancia:
            logger.info(f"   Convergência atingida na iteração {iteracao + 1}")
            break

        # Progresso a cada 10 iterações
        if (iteracao + 1) % 10 == 0:
            logger.info(f"   Iteração {iteracao + 1}: intervalo = [{left:.6f}, {right:.6f}]")

    # Threshold ótimo é o ponto médio do intervalo final
    threshold_otimo = (left + right) / 2
    score_otimo = funcao_objetivo_threshold(threshold_otimo, tuple(y_true), tuple(y_proba),
                                            peso_recall, peso_precision, peso_experiencia)

    tempo_execucao = time.time() - inicio_tempo

    logger.info("Busca Ternária Concluída:")
    logger.info(f"   Tempo de execução: {tempo_execucao:.3f} segundos")
    logger.info(f"   Iterações realizadas: {len(historico)}")
    logger.info(f"   Threshold ótimo: {threshold_otimo:.6f}")
    logger.info(f"   Score ótimo: {score_otimo:.6f}")

    return threshold_otimo, score_otimo, historico

# ============================================================================
# 4. EXECUÇÃO DA OTIMIZAÇÃO DE THRESHOLD
# ============================================================================

# Executando busca ternária com pesos de negócio customizados
threshold_otimo, score_otimo, historico_convergencia = busca_ternaria_otimizada(
    y_true, probabilidades,
    peso_recall=0.65,
    peso_precision=0.25,
    peso_experiencia=0.10,
    tolerancia=1e-8,
    max_iteracoes=150
)

# ============================================================================
# 5. COMPARAÇÃO: THRESHOLD PADRÃO VS OTIMIZADO
# ============================================================================

logger.info("Comparando Performance: Threshold Padrão vs Otimizado")

# Métricas com threshold padrão (0.5)
metricas_padrao = calcular_metricas_negocio(0.5, y_true, probabilidades)

# Métricas com threshold otimizado
metricas_otimizado = calcular_metricas_negocio(threshold_otimo, y_true, probabilidades)

# Criando DataFrame comparativo
comparacao_df = pd.DataFrame({
    'Threshold Padrão (0.5)': [
        metricas_padrao['precision'],
        metricas_padrao['recall'],
        metricas_padrao['f1_score'],
        metricas_padrao['fpr'],
        metricas_padrao['fnr']
    ],
    'Threshold Otimizado': [
        metricas_otimizado['precision'],
        metricas_otimizado['recall'],
        metricas_otimizado['f1_score'],
        metricas_otimizado['fpr'],
        metricas_otimizado['fnr']
    ]
}, index=['Precision', 'Recall', 'F1-Score', 'Taxa FP', 'Taxa FN'])

logger.info("\nComparação Detalhada:")
logger.info("\n" + str(comparacao_df.round(4)))

# Calculando melhorias percentuais
melhorias = {}
for metrica in ['precision', 'recall', 'f1_score']:
    valor_padrao = metricas_padrao[metrica]
    valor_otimizado = metricas_otimizado[metrica]
    if valor_padrao > 0:
        melhoria = ((valor_otimizado - valor_padrao) / valor_padrao) * 100
        melhorias[metrica] = melhoria

logger.info("Melhorias Alcançadas:")
logger.info(f"   Recall: {melhorias['recall']:+.1f}%")
logger.info(f"   Precision: {melhorias['precision']:+.1f}%")
logger.info(f"   F1-Score: {melhorias['f1_score']:+.1f}%")

# ============================================================================
# 6. VISUALIZAÇÕES ESPECÍFICAS DO EXEMPLO 2
# ============================================================================

# Configuração para plots
plt.style.use('default')
plt.rcParams['figure.figsize'] = [15, 12]

# Criando subplots para múltiplas visualizações
fig = plt.figure(figsize=(16, 12))

# PLOT 1: Convergência da Busca Ternária (agora em ChartJS, renderizado separadamente)
# JSON para ChartJS
chartjs_data = {
    "type": "line",
    "data": {
        "labels": list(range(1, len(historico_convergencia) + 1)),
        "datasets": [
            {
                "label": "Limite Inferior",
                "data": [h['left'] for h in historico_convergencia],
                "borderColor": "#1E90FF",
                "backgroundColor": "#1E90FF",
                "fill": False,
                "tension": 0.1
            },
            {
                "label": "Limite Superior",
                "data": [h['right'] for h in historico_convergencia],
                "borderColor": "#FF4500",
                "backgroundColor": "#FF4500",
                "fill": False,
                "tension": 0.1
            },
            {
                "label": f"Threshold Ótimo: {threshold_otimo:.4f}",
                "data": [threshold_otimo] * len(historico_convergencia),
                "borderColor": "#32CD32",
                "backgroundColor": "#32CD32",
                "borderDash": [5, 5],
                "fill": False,
                "tension": 0.1
            }
        ]
    },
    "options": {
        "plugins": {
            "title": {
                "display": True,
                "text": "Convergência da Busca Ternária",
                "font": {"size": 16, "weight": "bold"}
            },
            "legend": {"position": "top"}
        },
        "scales": {
            "x": {"title": {"display": True, "text": "Iteração"}},
            "y": {"title": {"display": True, "text": "Threshold"}, "min": 0, "max": 1}
        }
    }
}
logger.info("ChartJS para Convergência da Busca Ternária gerado (renderizar separadamente)")

# PLOT 2: Tamanho do Intervalo de Busca
ax2 = plt.subplot(2, 3, 2)
hist_df = pd.DataFrame(historico_convergencia)
ax2.semilogy(hist_df['iteracao'], hist_df['intervalo_size'], 'purple', linewidth=2, marker='o', markersize=4)
ax2.set_xlabel('Iteração')
ax2.set_ylabel('Tamanho do Intervalo (log)')
ax2.set_title('Redução do Intervalo de Busca', fontweight='bold', fontsize=12)
ax2.grid(True, alpha=0.3)

# PLOT 3: Função Objetivo vs Threshold (Análise de Sensibilidade)
ax3 = plt.subplot(2, 3, 3)
thresholds_teste = np.linspace(0.01, 0.99, 200)
scores_teste = [funcao_objetivo_threshold(t, tuple(y_true), tuple(y_proba), 0.65, 0.25, 0.10)
                for t in thresholds_teste]

ax3.plot(thresholds_teste, scores_teste, 'blue', linewidth=2)
ax3.axvline(x=threshold_otimo, color='red', linestyle='--', linewidth=2,
            label=f'Ótimo: {threshold_otimo:.4f}')
ax3.axvline(x=0.5, color='gray', linestyle=':', linewidth=2, label='Padrão: 0.5')
ax3.set_xlabel('Threshold')
ax3.set_ylabel('Score da Função Objetivo')
ax3.set_title('Análise de Sensibilidade', fontweight='bold', fontsize=12)
ax3.legend()
ax3.grid(True, alpha=0.3)

# PLOT 4: Comparação de Métricas (Bar Plot)
ax4 = plt.subplot(2, 3, 4)
x_pos = np.arange(len(comparacao_df.index))
width = 0.35

bars1 = ax4.bar(x_pos - width/2, comparacao_df['Threshold Padrão (0.5)'],
                width, label='Threshold Padrão', color='lightcoral', alpha=0.8)
bars2 = ax4.bar(x_pos + width/2, comparacao_df['Threshold Otimizado'],
                width, label='Threshold Otimizado', color='lightblue', alpha=0.8)

ax4.set_xlabel('Métricas')
ax4.set_ylabel('Valor')
ax4.set_title('Comparação de Performance', fontweight='bold', fontsize=12)
ax4.set_xticks(x_pos)
ax4.set_xticklabels(comparacao_df.index, rotation=45)
ax4.legend()
ax4.grid(axis='y', alpha=0.3)

# Adicionando valores nas barras
for bar in bars1:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height:.3f}', ha='center', va='bottom', fontsize=8)

for bar in bars2:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height:.3f}', ha='center', va='bottom', fontsize=8)

# PLOT 5: Curvas Precision-Recall e ROC
ax5 = plt.subplot(2, 3, 5)
precisions, recalls, fprs = [], [], []
thresholds_curva = np.linspace(0.01, 0.99, 100)

for t in thresholds_curva:
    metricas = calcular_metricas_negocio(t, y_true, probabilidades)
    precisions.append(metricas['precision'])
    recalls.append(metricas['recall'])
    fprs.append(metricas['fpr'])

ax5.plot(recalls, precisions, 'b-', linewidth=2, label='Curva Precision-Recall')
idx_otimo = np.argmin(np.abs(np.array(thresholds_curva) - threshold_otimo))
idx_padrao = np.argmin(np.abs(np.array(thresholds_curva) - 0.5))

ax5.plot(recalls[idx_otimo], precisions[idx_otimo], 'ro', markersize=10,
         label=f'Otimizado ({threshold_otimo:.3f})')
ax5.plot(recalls[idx_padrao], precisions[idx_padrao], 'go', markersize=10,
         label='Padrão (0.5)')
ax5.set_xlabel('Recall')
ax5.set_ylabel('Precision')
ax5.set_title('Curva Precision-Recall', fontweight='bold', fontsize=12)
ax5.legend()
ax5.grid(True, alpha=0.3)

# PLOT 6: Distribuição de Probabilidades
ax6 = plt.subplot(2, 3, 6)
prob_legitimas = probabilidades[y_true == 0]
prob_fraudes = probabilidades[y_true == 1]

ax6.hist(prob_legitimas, bins=50, alpha=0.6, label='Transações Legítimas',
         color='lightgreen', density=True)
ax6.hist(prob_fraudes, bins=50, alpha=0.6, label='Fraudes',
         color='lightcoral', density=True)

ax6.axvline(x=0.5, color='gray', linestyle=':', linewidth=2, label='Threshold Padrão')
ax6.axvline(x=threshold_otimo, color='red', linestyle='--', linewidth=2,
            label=f'Threshold Otimizado')
ax6.set_xlabel('Probabilidade de Fraude')
ax6.set_ylabel('Densidade')
ax6.set_title('Distribuição de Probabilidades', fontweight='bold', fontsize=12)
ax6.legend()
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================================================
# 7. SIMULAÇÃO DE ADAPTAÇÃO CONTÍNUA (CONCEPT DRIFT)
# ============================================================================

def simular_concept_drift_temporal(X_train, y_train, X_test, y_true):
    """
    Simula mudanças nos padrões de fraude ao longo do tempo
    e demonstra como o threshold deve ser continuamente re-otimizado
    """
    logger.info("Simulando Adaptação Contínua ao Concept Drift...")

    # Simulando 12 períodos mensais
    n_periodos = 12
    resultados_tempo = []
    modelo = RandomForestClassifier(
        n_estimators=200, max_depth=20, min_samples_split=5, random_state=42, n_jobs=-1
    )

    for mes in range(1, n_periodos + 1):
        logger.info(f"Processando Mês {mes}/12")

        # Simulando drift nas features
        drift_factor = 1.0 + 0.03 * mes
        seasonal_factor = 1.0 + 0.2 * np.sin(2 * np.pi * mes / 12)
        X_drift, y_drift = make_classification(
            n_samples=12000,
            n_features=15,
            n_informative=12,
            n_clusters_per_class=3,
            weights=[0.95 - 0.01 * mes, 0.05 + 0.01 * mes],  # Aumenta fraudes
            random_state=42 + mes,
            shift=drift_factor * 0.1,
            scale=seasonal_factor
        )

        # Re-treinar modelo para o período
        modelo.fit(X_train, y_train)
        prob_drift = modelo.predict_proba(X_drift)[:, 1]

        # Aplicando ruído nas probabilidades
        ruido = np.random.normal(1.0, 0.05, len(prob_drift))
        prob_drift = prob_drift * drift_factor * seasonal_factor * ruido
        prob_drift = np.clip(prob_drift, 0.001, 0.999)

        # Re-otimizando threshold
        threshold_mes, score_mes, _ = busca_ternaria_otimizada(
            y_drift, prob_drift,
            peso_recall=0.65, peso_precision=0.25, peso_experiencia=0.10,
            tolerancia=1e-6, max_iteracoes=50
        )

        # Calculando métricas
        metricas_mes = calcular_metricas_negocio(threshold_mes, y_drift, prob_drift)

        # Salvando resultados
        resultados_tempo.append({
            'mes': mes,
            'threshold': threshold_mes,
            'score': score_mes,
            'precision': metricas_mes['precision'],
            'recall': metricas_mes['recall'],
            'f1_score': metricas_mes['f1_score'],
            'fpr': metricas_mes['fpr'],
            'drift_factor': drift_factor,
            'seasonal_factor': seasonal_factor
        })

        logger.info(f"   Threshold: {threshold_mes:.4f}, F1: {metricas_mes['f1_score']:.3f}")

    return pd.DataFrame(resultados_tempo)

# Executando simulação
logger.info("Executando simulação de 12 meses...")
resultados_drift = simular_concept_drift_temporal(X_train, y_train, X_test, y_true)

# ============================================================================
# 8. VISUALIZAÇÃO DA ADAPTAÇÃO TEMPORAL
# ============================================================================

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Evolução do Threshold ao longo do tempo
ax1.plot(resultados_drift['mes'], resultados_drift['threshold'], 'bo-', linewidth=2, markersize=6)
ax1.axhline(y=0.5, color='gray', linestyle='--', alpha=0.7, label='Threshold Padrão')
ax1.set_xlabel('Mês')
ax1.set_ylabel('Threshold Ótimo')
ax1.set_title('Adaptação do Threshold ao Longo do Tempo', fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Plot 2: Performance (F1-Score) ao longo do tempo
ax2.plot(resultados_drift['mes'], resultados_drift['f1_score'], 'ro-', linewidth=2, markersize=6)
ax2.set_xlabel('Mês')
ax2.set_ylabel('F1-Score')
ax2.set_title('Performance Mantida com Adaptação', fontweight='bold')
ax2.grid(True, alpha=0.3)

# Plot 3: Fatores de Drift
ax3.plot(resultados_drift['mes'], resultados_drift['drift_factor'], 'g-',
         linewidth=2, label='Drift Linear')
ax3.plot(resultados_drift['mes'], resultados_drift['seasonal_factor'], 'orange',
         linewidth=2, label='Fator Sazonal')
ax3.set_xlabel('Mês')
ax3.set_ylabel('Fator de Multiplicação')
ax3.set_title('Fatores de Concept Drift Simulados', fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Trade-off Precision vs Recall
ax4.scatter(resultados_drift['recall'], resultados_drift['precision'],
           c=resultados_drift['mes'], cmap='viridis', s=100, alpha=0.7)
ax4.set_xlabel('Recall')
ax4.set_ylabel('Precision')
ax4.set_title('Trade-off Precision-Recall ao Longo do Tempo', fontweight='bold')
ax4.grid(True, alpha=0.3)

cbar = plt.colorbar(ax4.collections[0], ax=ax4)
cbar.set_label('Mês')

plt.tight_layout()
plt.show()

# ============================================================================
# 9. ANÁLISE DE COMPLEXIDADE ALGORÍTMICA
# ============================================================================

def analisar_complexidade_busca_ternaria():
    """
    Demonstra a eficiência da busca ternária comparada com busca linear
    """
    logger.info("Análise de Complexidade: Busca Ternária vs Linear")

    tamanhos_teste = [100, 1000, 10000, 100000]
    resultados_complexidade = []

    for n in tamanhos_teste:
        inicio = time.time()
        for _ in range(n):
            funcao_objetivo_threshold(0.5, tuple(y_true[:100]), tuple(probabilidades[:100]))
        tempo_linear = time.time() - inicio

        iteracoes_ternaria = int(np.log(n) / np.log(3)) + 1
        inicio = time.time()
        for _ in range(iteracoes_ternaria):
            funcao_objetivo_threshold(0.5, tuple(y_true[:100]), tuple(probabilidades[:100]))
        tempo_ternaria = time.time() - inicio

        resultados_complexidade.append({
            'n': n,
            'tempo_linear': tempo_linear,
            'tempo_ternaria': tempo_ternaria,
            'iteracoes_ternaria': iteracoes_ternaria,
            'speedup': tempo_linear / tempo_ternaria if tempo_ternaria > 0 else 0
        })

        logger.info(f"   n={n:6d}: Linear={tempo_linear:.4f}s, Ternária={tempo_ternaria:.4f}s, "
                    f"Speedup={tempo_linear/tempo_ternaria if tempo_ternaria > 0 else 0:.1f}x")

    return pd.DataFrame(resultados_complexidade)

complexidade_df = analisar_complexidade_busca_ternaria()

# ============================================================================
# 10. RELATÓRIO FINAL E INSIGHTS
# ============================================================================

logger.info("=" * 80)
logger.info("RELATÓRIO FINAL - EXEMPLO 2: OTIMIZAÇÃO CONTÍNUA DE THRESHOLD")
logger.info("=" * 80)

logger.info(f"""
PRINCIPAIS RESULTADOS:

Otimização de Threshold:
   • Threshold padrão: 0.500
   • Threshold otimizado: {threshold_otimo:.6f}
   • Melhoria F1-Score: {melhorias['f1_score']:+.1f}%
   • Melhoria Recall: {melhorias['recall']:+.1f}%

Eficiência Algorítmica:
   • Busca ternária: O(log₃ n) iterações
   • Convergência em {len(historico_convergencia)} iterações
   • Speedup médio: {complexidade_df['speedup'].mean():.1f}x vs busca linear

Adaptação Contínua:
   • Simulação de 12 meses com concept drift
   • Threshold varia de {resultados_drift['threshold'].min():.3f} a {resultados_drift['threshold'].max():.3f}
   • Performance F1 mantida em {resultados_drift['f1_score'].mean():.3f} ± {resultados_drift['f1_score'].std():.3f}

APLICAÇÕES PRÁTICAS:

1. Calibração Automática em Produção:
   • Re-otimização de threshold a cada período (diário/semanal)
   • Adaptação automática a mudanças nos padrões de fraude
   • Monitoramento contínuo de performance

2. Sistemas de Alerta Adaptativos:
   • Ajuste dinâmico baseado em feedback do negócio
   • Balanceamento automático entre detecção e experiência
   • Resposta rápida a novos tipos de fraude

3. A/B Testing de Thresholds:
   • Comparação de diferentes configurações em produção
   • Otimização baseada em métricas de negócio específicas
   • Rollback automático em caso de degradação

CONCEITOS MATEMÁTICOS DEMONSTRADOS:

• Busca ternária para otimização de função unimodal
• Análise de convergência e complexidade algorítmica
• Otimização contínua com restrições de domínio
• Simulação de concept drift temporal
• Trade-off entre múltiplos objetivos conflitantes

MÉTRICAS FINAIS DE NEGÓCIO:

• Taxa de detecção otimizada: {metricas_otimizado['recall']:.1%}
• Qualidade das detecções: {metricas_otimizado['precision']:.1%}
• Taxa de falsos positivos: {metricas_otimizado['fpr']:.2%}
• Score F1 balanceado: {metricas_otimizado['f1_score']:.3f}

VANTAGENS DA ABORDAGEM:

✓ Adaptação automática a mudanças nos dados
✓ Eficiência computacional (complexidade logarítmica)
✓ Flexibilidade para diferentes pesos de negócio
✓ Monitoramento contínuo de performance
✓ Escalabilidade para grandes volumes de dados
""")

logger.info("Integração com Exemplo 1:")
logger.info("   • Exemplo 1: Otimização de hiperparâmetros (decisão discreta)")
logger.info("   • Exemplo 2: Otimização de threshold (decisão contínua)")
logger.info("   • Juntos: Sistema completo de otimização multi-nível")

logger.info("Aplicação Industrial:")
logger.info("   • Similar ao pipeline para calibração contínua")
logger.info("   • Adaptação em tempo real a concept drift")
logger.info("   • Otimização automática de trade-offs de negócio")

logger.info("Exemplo 2 concluído com sucesso!")
logger.info("Prof. Maurizio Prizzi - maurizio.prizzi@ceub.edu.br")
logger.info("Repositório: https://github.com/maurizioprizzi/otimizacao-aplicada-ciencia-dados")
logger.info("=" * 80)